### Reads in data frame (from pickle)
### Subsets data frame (one list per month)
### Pulls lyrics from mxm database given input dates and max rank (i.e. top 20, or top 50, or top 100)

### This all should work so long as you have the csv files (from google drive) in the right place with the right name, and you have the Billboard100DF in your working directory. All you should have to change is the first line (put your mxm_key), and then in the cell that starts #for y in xrange(2007,2016): (for finishing the 2000s), you should edit that top lie to reflect the dates you want to pull. You can also change max_rank to something other than 20 if you want to get lyrics for more or fewer of the list.

In [30]:
# IMPORTANT: edit this cell with your API info for billboard and spotify
mxm_key = "f8a009fe01e53de1e8a1388e60812ef5" #swuerth

In [2]:
# Import everything we'll need in this notebook here
import billboard
from billboard import ChartData
import pdb 
import numpy as np 
import pandas as pd

# for lyrics stuff (mxm)
from musixmatch.track import TracksCollection
from musixmatch.lyrics import Lyrics

# for fuzzy matching artist names (when matching hot100 song to lyrics in mxm)
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# for joining list of strings
import string


# for ascii / utf8 errors, run this
import sys
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
print 'testing!'

import glob

import datetime

import csv


testing!


In [3]:
#before we read in the dataframe, let's get a list of dates included in the billboard dataset
#note this will only work if you have all the csv files that steph posted to google drive

file_list = glob.glob("./hot100/hot100*csv") #or wherever your csvs are stored
print 'num lists = ', len(file_list)

#
all_dates = [] #here we will store all the dates
date_ind_start = 16
date_ind_end = 26
for f in file_list:
    all_dates.append(f[date_ind_start:date_ind_end])
print 'first 4 dates are : ', all_dates[0:4]

# make another list 'monthly_dates' that only stores every 4th date from all_dates
monthly_dates = all_dates[0::4]
print 'subsetting to monthly gives ', len(monthly_dates), ' lists'
print 'first 4 dates of monthly subset are : ', monthly_dates[0:4]

#save monthly datelist to csv:
myfile = open('monthly_date_list.csv', 'wb')
wr = csv.writer(myfile)
wr.writerow(monthly_dates)

num lists =  3023
first 4 dates are :  ['1958-08-09', '1958-08-16', '1958-08-23', '1958-08-30']
subsetting to monthly gives  756  lists
first 4 dates of monthly subset are :  ['1958-08-09', '1958-09-06', '1958-10-04', '1958-11-01']


In [4]:
#read df from pickle file
df = pd.read_pickle('./Billboard100DF')

In [5]:
# we'll work with only month subset of full billboard df
## let's subset to monthly data using our monthly date list.
r = len(df)
print 'rows in full dataframe = ', r
# replace 'df' with only monthly lists
df = df[df['date'].isin(monthly_dates)].reindex()
print 'rows in monthly subset = ', len(df)
print 'ratio = ', len(df)/float(r)
df.head()

rows in full dataframe =  302142
rows in monthly subset =  75563
ratio =  0.250091016807


,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
0,Ricky Nelson,Re-Entry,0,1,1,33FPsMEl3UwpytDuyf9VYq,https://embed.spotify.com/?uri=spotify:track:3...,Poor Little Fool,11,1958-08-09
1,Perez Prado And His Orchestra,Re-Entry,0,2,2,0nG4SbrAONIFLIwQqegooj,https://embed.spotify.com/?uri=spotify:track:0...,Patricia,14,1958-08-09
2,Bobby Darin,Re-Entry,0,3,3,6NCPbpsr2N4w6YIZkpq6jx,https://embed.spotify.com/?uri=spotify:track:6...,Splish Splash,8,1958-08-09
3,Elvis Presley With The Jordanaires,Re-Entry,0,4,4,0MLachoG7wHVRrVydw2xuJ,https://embed.spotify.com/?uri=spotify:track:0...,Hard Headed Woman,8,1958-08-09
4,Kalin Twins,Re-Entry,0,5,5,52l2PCOwU2n2zfTlc0pEov,https://embed.spotify.com/?uri=spotify:track:5...,When,9,1958-08-09


In [6]:
### Edit DataFrame such that rank, LostPos,PeakPos,Rank,Weeks are all integers ?
### Add numeric date fields

df['Weeks'] = pd.to_numeric(df['Weeks'])
df['Rank'] = pd.to_numeric(df['Rank'])
df['date'] = pd.to_datetime(df['date'])
# now that rank and date are in numeric form, we can subset with > & <
# for example
print '# songs in top 20 (all dates): ',len(df[df['Rank']<20])
print 'displaying just the parts from June 2016 onwards:'
df[df['date'] > datetime.date(year=2016,month=6,day=1)]

# songs in top 20 (all dates):  14355
displaying just the parts from June 2016 onwards:


,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
301842,Drake Featuring WizKid & Kyla,0,1,1,1,11hqMWwX7sF3sOGdtijofF,https://embed.spotify.com/?uri=spotify:track:1...,One Dance,10,2016-06-25
301843,Desiigner,0,2,1,2,6r2jK1A6oFRPREZfxjc5d1,https://embed.spotify.com/?uri=spotify:track:6...,Panda,16,2016-06-25
301844,Justin Timberlake,0,3,1,3,6JV2JOEocMgcZxYSZelKcc,https://embed.spotify.com/?uri=spotify:track:6...,Can't Stop The Feeling!,5,2016-06-25
301845,The Chainsmokers Featuring Daya,+1,5,3,4,1i1fxkWeaMmKEB4T7zqbzK,https://embed.spotify.com/?uri=spotify:track:1...,Don't Let Me Down,17,2016-06-25
301846,Fifth Harmony Featuring Ty Dolla $ign,-1,4,4,5,0SJPThTy7ynySPF4euczx7,https://embed.spotify.com/?uri=spotify:track:0...,Work From Home,15,2016-06-25
301847,Lukas Graham,0,6,2,6,7129iqBafaphfc3WPCGC0L,https://embed.spotify.com/?uri=spotify:track:7...,7 Years,21,2016-06-25
301848,Calvin Harris Featuring Rihanna,+2,9,4,7,0azC730Exh71aQlOt9Zj3y,https://embed.spotify.com/?uri=spotify:track:0...,This Is What You Came For,6,2016-06-25
301849,Mike Posner,-1,7,4,8,5M2j5k8uS08xE8HnGVgabR,https://embed.spotify.com/?uri=spotify:track:5...,I Took A Pill In Ibiza,21,2016-06-25
301850,Rihanna,-1,8,7,9,3kfxarilcBr81mb2hmZLeh,https://embed.spotify.com/?uri=spotify:track:3...,Needed Me,19,2016-06-25
301851,P!nk,0,10,10,10,7K5dzhGda2vRTaAWYI3hrb,https://embed.spotify.com/?uri=spotify:track:7...,Just Like Fire,8,2016-06-25


In [15]:
# make function that takes a list of artists (from mxm), and selects the best match to the artist from hot100
def find_artist_match(name_to_match, artist_list):
    index = 0 #index of the artist we're scoring
    r = 0 #here i'll store the highest similarity score
    best_match = 'none'
    best_ind = -1 #initialize
    for a in artist_list:
        rnew = fuzz.ratio(name_to_match, a)
        #print 'similarity between ', name_to_match, ' and ', a, ' : ', rnew
        if rnew > r: 
            best_match = a
            best_ind = index
            r = rnew
        index = index+1

    print 'Best Match = ', best_match
    assert(best_match==artist_list[best_ind])
    print 'Score = ',r
    return {'ind':best_ind, 'score':r} 


In [10]:
#want 2007 to present day
#for y in xrange(2007,2016)
for y in xrange(1992,2000):
    print y
    

1992
1993
1994
1995
1996
1997
1998
1999


## Here is where you will want to start getting data. 
### Make sure your mxm key is stored in mxm_key at the top of this notebook.
### In the block below, change the years in xrange() to bracket the dates you want to get top 20 lists from
### As of 6pm Thursday, Jully 14, Stephanie has done this for years 2000-2015 and for 1990-1998.
### In the 1990's we still need './Lyrics_1999-01-01-2000-01-01MaxRank20' to be made, and then we need older decades 1950s, 60s, 70s, 80s.


In [18]:
#for y in xrange(2007,2016): (for finishing the 2000s)
for y in xrange(1992,2000): #get the 90s!

    ## Initialize data frames and lists:
    apikey = mxm_key

    #header for info to be stored in df_mxm_info
    cols = ['lyrics_id', 'has_lyrics', 'artist_name', 'track_spotify_id','track_id','track_name']

    #header for info in df_mxm_lyrics
    lyrics_df_cols = ['lyrics_copyright', 'lyrics_body','publisher_list', 'can_edit', 'writer_list', \
                      'instrumental','verified','html_tracking_url','lyrics_language_description', \
                      'locked','restricted','explicit','updated_time','lyrics_id','script_tracking_url', \
                      'action_requested','pixel_tracking_url','lyrics_language']

    # initialize dfs and lists :
    #if 'df_mxm_info' not in locals():
    df_mxm_info = pd.DataFrame(columns = cols) #dataframe for some song info (from collections object)
    #if 'df_mxm_lyrics' not in locals():
    df_mxm_lyrics = pd.DataFrame(columns = lyrics_df_cols) #dataframe for lyrics and related attributes
    #if 'lyrics_list' not in locals():
    lyrics_list = [] #append lyric info to a list for each song, later we'll put it in a dataframe!
    #if 'hot_artists_no_lyric' not in locals():
    hot_artists_no_lyrics = [] #when we don't find lyrics, store artistname here
    #if 'hot_tracknames_no_lyrics' not in locals():
    hot_tracknames_no_lyrics = [] #when we don't find lyrics, store trackname here
    print 'df_mxm_info has ', len(df_mxm_info), ' rows' 
    print 'df_mxm_lyrics has ', len(df_mxm_info), ' rows' 
    print 'lyrics_list has length ', len(df_mxm_info)

    #subset df_monthly given start_date, end_date, and max_rank:
    start_date = datetime.date(year=y,month=1,day=1)
    end_date = datetime.date(year=y+1,month=1,day=1)
    max_rank = 20 #just top N songs

    df_reduced = df[(df['date'] > start_date) & (df['date']< end_date) & (df['Rank']<=max_rank)]

    print ' number songs in df_reduced = ', len(df_reduced)

    #name for saving df
    name2save = './Lyrics_'+str(start_date)+'-'+str(end_date)+'MaxRank'+str(max_rank)
    print 'will save df as : ', name2save


    #now get the lyrics

    for i in xrange(len(df_reduced)):
        trackname_hot = df_reduced['Title'].iloc[i]
        artist_hot = df_reduced['Artist'].iloc[i]
        print 'processing ', trackname_hot, ' by ', artist_hot

        #ask mxm for info about this track (search on trackname for now, more complex later?)
        collection = TracksCollection.fromSearch(q_track=trackname_hot, apikey = apikey)
        if len(collection)==0:
            print('No lyrics found')
            hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
            hot_tracknames_no_lyrics.append(trackname_hot) 
            continue # skip this row of df_reduced
        #store collection in dataframe called 'subset'
        subset = pd.DataFrame.from_dict(collection)[cols]
        #filter for only songs with lyrics:
        subset = subset[subset['has_lyrics']==1]
        #move on if no matches found...
        if len(subset)==0:
            print ' No lyrics found ...'
            hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
            hot_tracknames_no_lyrics.append(trackname_hot) 
            continue # skip this row of df_reduced -- in future we should try other searches when this initial search fails
        artists = subset['artist_name'].values
        print 'There are ', len(artists), ' artists with this song title'
        track_id = subset['track_id'].values

        # fuzzy match the artist from hot100 to all artists matching track name
        try:
            result = find_artist_match(artist_hot, artists)
        except AssertionError:
            continue
        ind = result['ind']
        score = result['score']
        print 'matched artist name = ', artists[ind]
        #return {'ind':best_ind, 'score':r} 

        # select the best track id (best match to artist name)
        track_id_selected = track_id[ind]
        # print track_id_selected

        # now get the lyrics, but only if the 'track_id' is not already in the info df
        if track_id_selected in df_mxm_info.track_id.values:
            print ' already got lyrics for artist, track_id = ', track_id_selected
        if not track_id_selected in df_mxm_info.track_id.values:
            # Get Lyrics!
            try:
                lyrics = Lyrics(track_id = track_id_selected, apikey = apikey)
                # Append lyrics to list
                # lyrics_vals = [lyrics.values()]
                #this is inefficint, list will keep growing and growing! fix at some point
                lyrics_list.append(lyrics.values())

                # subset[is_true] is info for the best match to the billboard title+artist 
                is_true = subset['track_id'] == track_id_selected     
                add_to_mxm_info = subset[is_true]
                #add col with hot100 artist name, and one with hot100 song name
                add_to_mxm_info['hot100 artist'] = artist_hot
                add_to_mxm_info['hot100 track'] = trackname_hot
                add_to_mxm_info['artist string match score'] = result['score']

                # append new song info to df_mxm_info
                df_mxm_info = df_mxm_info.append(add_to_mxm_info)
                #check that lyrics list is same length as info df
                # assert (len(lyrics_list) == len(df_mxm_info))
                df_mxm_lyrics = pd.DataFrame(data= lyrics_list, columns=lyrics_df_cols)
                # df_mxm_lyrics_tmp = pd.DataFrame(data= lyrics_vals, columns=lyrics_df_cols)
                # df_mxm_lyrics = df_mxm_lyrics.append(df_mxm_lyrics_tmp)
                # merge these 2 dataframes
                # note that the song with no lyrics is exclued from the final data frame. this makes sense?
                df_mxm = pd.merge(df_mxm_info,df_mxm_lyrics,on='lyrics_id')
            except : 
                print 'no lyrics for track id', track_id_selected
                hot_artists_no_lyrics.append(artist_hot) #when we don't find lyrics, store artistname here
                hot_tracknames_no_lyrics.append(trackname_hot) 
                pass

    # Save the data frame!
    df_mxm.to_pickle(name2save)
    # Also save lists of no match:
    if len(hot_artists_no_lyrics)>0:
        #save monthly datelist to csv:
        myfile = open('NoLyricsList_'+ str(start_date)+'-'+str(end_date)+'MaxRank'+str(max_rank)+'.csv', 'wb')
        wr = csv.writer(myfile)
        wr.writerow('Artists with no lyrics found:')
        wr.writerow(hot_artists_no_lyrics)
        wr.writerow('Corresponding track names:')
        wr.writerow(hot_tracknames_no_lyrics)

df_mxm_info has  0  rows
df_mxm_lyrics has  0  rows
lyrics_list has length  0
 number songs in df_reduced =  260
will save df as :  ./Lyrics_1992-01-01-1993-01-01MaxRank20
processing  Black Or White  by  Michael Jackson
There are 

/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/IPython/kernel/__main__.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/IPython/kernel/__main__.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/IPython/kernel/__main__.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

 10  artists with this song title
Best Match =  Michael Jackson
Score =  100
matched artist name =  Michael Jackson
processing  All 4 Love  by  Color Me Badd
There are  6  artists with this song title
Best Match =  Color Me Badd
Score =  100
matched artist name =  Color Me Badd
processing  Can t Let Go  by  Mariah Carey
There are  9  artists with this song title
Best Match =  Mariah Carey
Score =  100
matched artist name =  Mariah Carey
processing  It s So Hard To Say Goodbye To Yesterday  by  Boyz II Men
There are  7  artists with this song title
Best Match =  Boyz II Men
Score =  100
matched artist name =  Boyz II Men
processing  2 Legit 2 Quit  by  M.C. Hammer
There are  8  artists with this song title
Best Match =  MC Hammer
Score =  90
matched artist name =  MC Hammer
processing  Smells Like Teen Spirit  by  Nirvana
There are  10  artists with this song title
Best Match =  Nirvana
Score =  100
matched artist name =  Nirvana
processing  Addams Groove  by  M.C. Hammer
There are  2  

Error: Authentication failed, probably because of a bad API key.

### These next few cells are just checking the most recent data frame to see how it looks.

In [20]:
hot_artists_no_lyrics

[]

In [22]:
df_reduced.tail()

,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
215457,Ideal,-2,14,13,16,0CofintZCm8MhxiOMrauiT,https://embed.spotify.com/?uri=spotify:track:0...,Get Gone,22,1999-12-04
215458,Blaque,+2,19,5,17,,,Bring It All To Me,29,1999-12-04
215459,Britney Spears,-1,17,10,18,1DSJNBNhGZCigg9ll5VeZv,https://embed.spotify.com/?uri=spotify:track:1...,(You Drive Me) Crazy,20,1999-12-04
215460,Sugar Ray,-1,18,7,19,6NcHJ23mLQKnVxQcOmT7JW,https://embed.spotify.com/?uri=spotify:track:6...,Someday,32,1999-12-04
215461,Juvenile Featuring Mannie Fresh,+1,21,19,20,6o2g1BJvtYQssH84kBYs7y,https://embed.spotify.com/?uri=spotify:track:6...,Back That Thang Up,30,1999-12-04


In [23]:
cols_to_view = ['artist string match score','artist_name','hot100 artist','track_name','lyrics_body']
df_mxm[cols_to_view].tail()

,artist string match score,artist_name,hot100 artist,track_name,lyrics_body
28,44.0,Alabama feat. 'N Sync,N Sync,God Must Have Spent a Little More Time On You,Can this be true? Tell me can this be real?\nH...
29,33.0,Go Go Berlin,Jesse Powell,You You You,"Yeah this one thing, that I need\nIs for you t..."
30,100.0,Case,Case,Faded Pictures (feat. Joe),"That's right\nLife is funny, ain't it?\nSometi..."
31,100.0,Gerald Levert,Gerald Levert,Taking Everything,"To my heart\nTo my car\nTo my crib\nMy kids, e..."
32,38.0,Labbayk,Sugar Ray,Every Morning,Home > Labbayk > Every Morning\n\nEvery mornin...


In [24]:
df_mxm[df_mxm['artist_name']=='Nickelback']
df_mxm[df_mxm['track_id']==82751560][['lyrics_body']].values

array([], shape=(0, 1), dtype=object)

In [25]:
len(df_mxm)

33

In [26]:
df_mxm['lyrics_body'].values[0:10]

array([ u"No mountain's too high for you to climb\nAll you have to do is have some climbing faith, oh yeah\nNo river's too wide for you to make it across\nAll you have to do is believe it when you pray\nAnd then you will see the morning will come\nAnd every day will be bright as the sun\nAll of your fears cast them on me\nI just want you to see\n\nI'll be your cloud up in the sky\nI'll be your shoulder when you cry\nI hear your voices when you call me\nI am your angel\nAnd when all hope is gone, I'm here\nNo matter how far you are, I'm near\nIt makes no difference who you are\nI am your angel\n\nI'm your angel\n\nI saw your teardrops and I heard you cry\nAll you need is time seek me and you shall find\nYou have everything and you're still lonely\nIt don't have to be this way\nLet me show you a better day\n...\n\n******* This Lyrics is NOT for Commercial use *******",
       u'How did you get here? Nobody\'s supposed to be here\nI\'ve tried that love thing for the last time\nMy heart sa

## note: the fuzzy matching can sometimes pull the wrong lyrics, and it's not clear what cutoff string matching score should be, for example:
### Low score but true match:
processing  This I Promise You  by  'N Sync

There are  10  artists with this song title

Best Match =  \*NSYNC

Score =  31 
### Low score, false match:
processing  I Wish  by  R. Kelly

There are  10  artists with this song title

Best Match =  One Direction

Score =  19

### Higher score, false match:
processing  Stronger  by  Britney Spears
There are  9  artists with this song title
Best Match =  Kanye West
Score =  42
matched artist name =  Kanye West

### Should think more about this later, but for now I'm not cutting anything off and instead have a column in lyrics section for artist and trackname of the hot100 track used to find these lyrics
